## **1. Install and import libraries**

In [1]:
!pip install -qq datasets==2.16.1 evaluate==0.4.1 transformers[sentencepiece]==4.35.2
!pip install -qq accelerate==0.26.1
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [2]:
import os
import math
import torch
import pandas as pd

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from huggingface_hub import notebook_login
from datasets import Dataset

In [2]:
notebook_login()

## **2. Download and load poem dataset**

In [3]:
# https://drive.google.com/file/d/1KfrBAycsgQBt1mtEbzJh5pSYL8YIk0Tc/view?usp=sharing
!gdown --id 1KfrBAycsgQBt1mtEbzJh5pSYL8YIk0Tc
!unzip poem_dataset_final.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1KfrBAycsgQBt1mtEbzJh5pSYL8YIk0Tc
To: /content/poem_dataset_final.zip
100% 37.6k/37.6k [00:00<00:00, 90.4MB/s]
Archive:  poem_dataset_final.zip
  inflating: poem_final.csv          


In [4]:
DATASET_PATH = 'poem_final.csv'
df = pd.read_csv(DATASET_PATH)
df

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
2,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
3,3,"“Đến, nhiều nơi để đến”","Đến, nhiều nơi để đến\nVề, trở lại với mình\nC...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Đừng bao giờ dại dột”,Đừng bao giờ dại dột\nĐem chuyện riêng của mìn...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...,...
185,95,Ám ảnh sông xưa,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
186,96,Áng dương không biết sầu,Áng dương không biết sầu\nNằm mãi ở trên cao\n...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
187,97,Anh,Cây bút gẫy trong tay\nCặn mực khô đáy lọ\nÁnh...,19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...
188,98,Anh biết,Không có anh để già\nLàm sao em được trẻ\nMuốn...,NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [8]:
print(df['content'][0])

Cái làm ta hạnh phúc
Thực ra cũng chẳng nhiều
Chỉ cần có ai đó
Để ta thầm thương yêu

Rồi thêm chút công việc
Cho ta làm hàng ngày
Cuối cùng, chút mơ mộng
Để đưa ta lên mây


In [10]:
def split_content(content):
    samples = []

    poem_parts = content.split('\n\n')
    for poem_part in poem_parts:
        poem_in_lines = poem_part.split('\n')
        if len(poem_in_lines) == 4:
            samples.append(poem_in_lines)

    return samples

df['content'] = df['content'].apply(lambda x: split_content(x))
df

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,"[[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhi...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Chiều vừa xốp trên tay”,"[[Chiều vừa xốp trên tay, Chợt nghe thoáng ong...","Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
2,2,“Dưới giàn hoa thiên lý...”,"[[Dưới giàn hoa thiên lý, Một mình anh đang ng...","Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
3,3,"“Đến, nhiều nơi để đến”","[[Đến, nhiều nơi để đến, Về, trở lại với mình,...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Đừng bao giờ dại dột”,"[[Đừng bao giờ dại dột, Đem chuyện riêng của m...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...,...
185,95,Ám ảnh sông xưa,"[[Ôi, con sóng chết khô,, vật vờ trong bùn quá...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
186,96,Áng dương không biết sầu,"[[Áng dương không biết sầu, Nằm mãi ở trên cao...","Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
187,97,Anh,[],19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...
188,98,Anh biết,"[[Không có anh để già, Làm sao em được trẻ, Mu...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [11]:
df_exploded = df.explode('content')
df_exploded.reset_index(drop=True, inplace=True)
df_exploded = df_exploded.dropna(subset=['content'])
df_exploded

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,"[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhiề...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,0,“Cái làm ta hạnh phúc”,"[Rồi thêm chút công việc, Cho ta làm hàng ngày...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,1,“Chiều vừa xốp trên tay”,"[Chiều vừa xốp trên tay, Chợt nghe thoáng ong ...","Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,1,“Chiều vừa xốp trên tay”,"[Ớt đỏ sao cứ đỏ, Táo chín cho thật vàng, Em đ...","Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
4,2,“Dưới giàn hoa thiên lý...”,"[Dưới giàn hoa thiên lý, Một mình anh đang ngồ...","Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
...,...,...,...,...,...
481,98,Anh biết,"[Đất hiến dâng tuổi trẻ, Trời buông thả tuổi g...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
482,99,Anh bốn mùa lập đông,"[Buồn, thì buồn thiệt đó, Em à, có biết không,...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
483,99,Anh bốn mùa lập đông,"[Vâng, phiến buồn còn đó, Em nào biết phải khô...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
484,99,Anh bốn mùa lập đông,"[Ừ, nỗi buồn vậy đó, Em đừng vội bâng khuâng, ...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [12]:
df_exploded['content'] = df_exploded['content'].apply(lambda x: '\n'.join(x))
df_exploded

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,0,“Cái làm ta hạnh phúc”,Rồi thêm chút công việc\nCho ta làm hàng ngày\...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,1,“Chiều vừa xốp trên tay”,Ớt đỏ sao cứ đỏ\nTáo chín cho thật vàng\nEm đẹ...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
4,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
...,...,...,...,...,...
481,98,Anh biết,Đất hiến dâng tuổi trẻ\nTrời buông thả tuổi gi...,NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
482,99,Anh bốn mùa lập đông,"Buồn, thì buồn thiệt đó\nEm à, có biết không\n...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
483,99,Anh bốn mùa lập đông,"Vâng, phiến buồn còn đó\nEm nào biết phải khôn...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
484,99,Anh bốn mùa lập đông,"Ừ, nỗi buồn vậy đó\nEm đừng vội bâng khuâng\nV...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [13]:
poem_dataset = Dataset.from_pandas(df_exploded)
poem_dataset

Dataset({
    features: ['Unnamed: 0', 'title', 'content', 'source', 'url', '__index_level_0__'],
    num_rows: 441
})

In [14]:
TEST_SIZE = 0.1
poem_dataset = poem_dataset.train_test_split(test_size=TEST_SIZE)
poem_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'content', 'source', 'url', '__index_level_0__'],
        num_rows: 396
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'content', 'source', 'url', '__index_level_0__'],
        num_rows: 45
    })
})

## **3. Preprocess dataset**

In [15]:
MODEL_NAME = 'danghuy1999/gpt2-viwiki'

tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/773k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/431k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

In [16]:
tokenizer.pad_token = tokenizer.eos_token
MAX_SEQ_LEN = 100

def preprocess_function(row):
    return tokenizer(
        row['content'],
        max_length=MAX_SEQ_LEN,
        padding='max_length',
        truncation=True
    )

tokenized_poem_dataset = poem_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=poem_dataset['train'].column_names,
)

Map (num_proc=4):   0%|          | 0/396 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/45 [00:00<?, ? examples/s]

In [12]:
tokenizer('xin chào các bạn', max_length=10, padding='max_length', truncation=True)

{'input_ids': [6504, 3939, 331, 1486, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]}

In [13]:
tokenized_poem_dataset['train'][0].keys()

dict_keys(['input_ids', 'attention_mask'])

In [25]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

## **4. Training**

In [36]:
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

In [37]:
training_args = TrainingArguments(
    output_dir='gpt2_viet_poem_generation',
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=True
)

In [38]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_poem_dataset['train'],
    eval_dataset=tokenized_poem_dataset['test'],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

Step,Training Loss
500,5.629400
1000,4.272700


TrainOutput(global_step=1000, training_loss=4.95109423828125, metrics={'train_runtime': 339.7734, 'train_samples_per_second': 23.31, 'train_steps_per_second': 2.943, 'total_flos': 404186112000000.0, 'train_loss': 4.95109423828125, 'epoch': 20.0})

In [18]:
trainer.push_to_hub(commit_message="Training complete")

CommitInfo(commit_url='https://huggingface.co/thangduong0509/gpt2_viet_poem_generation/commit/c474a475f2fdb00f14973b51c0e168f978c31d5e', commit_message='Training complete', commit_description='', oid='c474a475f2fdb00f14973b51c0e168f978c31d5e', pr_url=None, pr_revision=None, pr_num=None)

## **5. Inference**

In [20]:
from transformers import pipeline

prompt = 'Con sông quê tôi đẹp\n'
generator = pipeline('text-generation', model='thangduong0509/gpt2_viet_poem_generation')
results = generator(
    prompt,
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    repetition_penalty=1.2
)
results = results[0]['generated_text']
print()
for line in results.split('\n'):
    print(line)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Con sông quê tôi đẹp
Chây xa đâu nắng!
Người bạn tình yêu…
Một cách nào nào thì?
Có thể đến rất nhiều!
Nên đất nước nhỏ… còn nhỏ?
Làm thật là thế!
Đời mình


In [126]:
prompt = 'Học học nữa học mãi\n'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
outputs = model.generate(
    inputs,
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    repetition_penalty=1.2
)
results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
results = results[0]
print()
for line in results.split('\n'):
    print(line)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Học học nữa học mãi
Hàng trăm ngàn đêm dài
Nhưng, không ai quên
Quanh ta thấy chuyện? đỏ?
Tôi thấy mình là học giả!
Đôi hay bạc đau may chờ.
Còn để ra mắt nào nữa?
Vì, ta thấy bạn
